## **DATA PREPARATION** 





# 1. SOURCES AND LIBRARIES

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
from pandasql import sqldf

# 2. LOADING DATA 

The goal of this section is to obtain datasets that can be cleaned efficiently afterwards and that will form a base for the model creation. Especially :     
- a 360 order model
- a 360 contact model

## Loading

In [ ]:
offline_contacts = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_technique_cartelis/offline_contacts.csv')
offline_orders = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_technique_cartelis/offline_orders.csv')
online_consent = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_technique_cartelis/online_consent.csv')
online_orders = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_technique_cartelis/online_orders.csv')
online_contacts = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_technique_cartelis/online_contacts.csv')

# 3. 360 MODELS


In this section I'll create 360 models (for contacts and orders) and try to keep as much customer informations as I can.

### 360 contacts

#### Quick check of duplicates

Let's see if in the contacts table provided we don't have duplicate informations before manipulating it :

In [ ]:
verif_contacts_offline = """
select
  *
  , count(*) as count
from offline_contacts
group by email
having count > 1
"""
sqldf(verif_contacts_offline, globals())

,id,email,firstname,lastname,civility,birth,mobile,address,zip,created,count


In [ ]:
verif_contacts_online = """
select
  *
  , count(*) as count
from online_contacts
group by email
having count > 1
"""
sqldf(verif_contacts_online, globals())

,email,birth,mobile,created,count


It's clean, let's advance.

#### Union online and offline contact and remove duplicates

First of all, we can union online and offline contacts. 

We create a channel field to differenciate offline orders and online orders.

In [ ]:
contacts = """
select
  email
  , firstname
  , lastname
  , civility
  , birth
  , mobile
  , address
  , zip
  , created
  , 'offline' as channel
from offline_contacts

union

select
  email
  , null as firstname
  , null as lastname
  , null as civility
  , birth
  , mobile
  , null as address
  , null as zip
  , created
  , 'online' as channel
from online_contacts


"""
sqldf(contacts, globals())

,email,firstname,lastname,civility,birth,mobile,address,zip,created,channel
0,a-ethan.blanchard620@hotmail.fr,None,None,None,NaN,3.365230e+10,None,NaN,2019-08-08 14:54:33.270,offline
1,a-evan.roux29452@gmail.com,None,None,None,NaN,NaN,None,NaN,2019-04-25 22:05:31.708,offline
2,a-juliette.lopez61584@hotmail.fr,None,None,None,NaN,NaN,None,NaN,2017-07-18 14:35:34.433,online
3,a-juliette.lopez61584@hotmail.fr,None,None,m,NaN,NaN,None,NaN,2017-07-18 14:35:34.433,offline
4,a-noah.fournier506@yahoo.fr,None,None,mlle,NaN,6.527320e+08,None,NaN,2015-06-01 22:32:18.820,offline
...,...,...,...,...,...,...,...,...,...,...
18627,zoe_robert59@laposte.net,zoe,robert,None,NaN,3.369949e+10,105 cours de meunier molière,89703.0,2019-10-22 09:45:25.102,offline
18628,zoe_roussel594@numericable.fr,zoe,roussel,None,NaN,NaN,None,NaN,2016-05-13 18:27:32.473,offline
18629,zoe_simon507@neuf.fr,None,None,None,19690623.0,NaN,None,NaN,2019-04-30 14:35:02.980,online
18630,zoe_vincent356@numericable.fr,None,None,None,19910421.0,NaN,None,NaN,2019-10-07 07:27:29.997,online


In [ ]:
df_contacts = sqldf(contacts, globals())

In [ ]:
offline_contacts.isna().sum()

id              0
email           0
firstname    2225
lastname     2225
civility     6018
birth        5085
mobile       7256
address      8769
zip          8769
created         0
dtype: int64

We can see that a customer contact is sometime describe by multiple rows (that's the case for zoe vincent for instance) because some customers are regitered offline **and** online. 

To deal with that and keep a unique row to describe each customer (the one with more informations) we use the following query that clean the dataset :

In [ ]:
contacts_clean = """
select distinct
  email
  , max(firstname) as firstname
  , max(lastname) as lastname
  , max(civility) as civility
  , max(birth) as birth
  , max(mobile) as mobile
  , max(address) as address
  , max(zip) as zip
  , max(created) as created
  , max(channel) as channel
  , case when (count(*)>1) then 'yes'
         else 'no' 
         end as is_client_using_both_channel
from df_contacts
group by 1
"""
sqldf(contacts_clean, globals())

,email,firstname,lastname,civility,birth,mobile,address,zip,created,channel,is_client_using_both_channel
0,a-ethan.blanchard620@hotmail.fr,None,None,None,NaN,3.365230e+10,None,NaN,2019-08-08 14:54:33.270,offline,no
1,a-evan.roux29452@gmail.com,None,None,None,NaN,NaN,None,NaN,2019-04-25 22:05:31.708,offline,no
2,a-juliette.lopez61584@hotmail.fr,None,None,m,NaN,NaN,None,NaN,2017-07-18 14:35:34.433,online,yes
3,a-noah.fournier506@yahoo.fr,None,None,mlle,NaN,6.527320e+08,None,NaN,2015-06-01 22:32:18.820,offline,no
4,a.chlo.muller29774@hotmail.fr,None,None,Mr,19840619.0,NaN,None,NaN,2019-05-30 01:01:19.406,online,yes
...,...,...,...,...,...,...,...,...,...,...,...
14995,zoe_remy383@yahoo.fr,None,None,None,19610809.0,3.364086e+10,None,NaN,2019-07-04 11:09:56.327,online,no
14996,zoe_robert59@laposte.net,zoe,robert,None,NaN,3.369949e+10,105 cours de meunier molière,89703.0,2019-10-22 09:45:25.102,offline,no
14997,zoe_roussel594@numericable.fr,zoe,roussel,None,NaN,NaN,None,NaN,2016-05-13 18:27:32.473,offline,no
14998,zoe_simon507@neuf.fr,None,None,None,19690623.0,NaN,None,NaN,2019-04-30 14:35:02.980,online,no


In [ ]:
df_contacts_clean = sqldf(contacts_clean, globals())

We can enriched contacts with online_consent using the email address :

In [ ]:
contacts_and_consent = """
select
  *
from df_contacts_clean
left join online_consent
using (email)
"""
sqldf(contacts_and_consent, globals())

,email,firstname,lastname,civility,birth,mobile,address,zip,created,channel,is_client_using_both_channel,optin
0,a-ethan.blanchard620@hotmail.fr,None,None,None,NaN,3.365230e+10,None,NaN,2019-08-08 14:54:33.270,offline,no,NaN
1,a-evan.roux29452@gmail.com,None,None,None,NaN,NaN,None,NaN,2019-04-25 22:05:31.708,offline,no,NaN
2,a-juliette.lopez61584@hotmail.fr,None,None,m,NaN,NaN,None,NaN,2017-07-18 14:35:34.433,online,yes,0.0
3,a-noah.fournier506@yahoo.fr,None,None,mlle,NaN,6.527320e+08,None,NaN,2015-06-01 22:32:18.820,offline,no,NaN
4,a.chlo.muller29774@hotmail.fr,None,None,Mr,19840619.0,NaN,None,NaN,2019-05-30 01:01:19.406,online,yes,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14995,zoe_remy383@yahoo.fr,None,None,None,19610809.0,3.364086e+10,None,NaN,2019-07-04 11:09:56.327,online,no,0.0
14996,zoe_robert59@laposte.net,zoe,robert,None,NaN,3.369949e+10,105 cours de meunier molière,89703.0,2019-10-22 09:45:25.102,offline,no,NaN
14997,zoe_roussel594@numericable.fr,zoe,roussel,None,NaN,NaN,None,NaN,2016-05-13 18:27:32.473,offline,no,NaN
14998,zoe_simon507@neuf.fr,None,None,None,19690623.0,NaN,None,NaN,2019-04-30 14:35:02.980,online,no,0.0


In [ ]:
df_contacts_and_consent = sqldf(contacts_and_consent, globals())
df_contacts_and_consent.optin.isna()

0         True
1         True
2        False
3         True
4        False
         ...  
14995    False
14996     True
14997     True
14998    False
14999    False
Name: optin, Length: 15000, dtype: bool

Here we have 3 possible values for optin : 
- NaN : no information (offline is not concerned)
- 0 : customer did not give his consent
- 1 : customer gave his consent 

### 360 orders

Let's have a 360 order model regrouping offline and online orders.

#### Offline order enrichment

First, we need to enrich the offline order table with some offline contact information such as the email.

In [ ]:
offline_orders_enriched = """
select
  o.id
  , o.contact_id as client_id
  , c.email
  , store	
  , quantity	
  , discount	
  , total_price	
  , o.created
from offline_orders  as o
left join offline_contacts as c
  on o.contact_id = c.id
"""
sqldf(offline_orders_enriched, globals())

,id,client_id,email,store,quantity,discount,total_price,created
0,s_4a7b9ff1-2074-4a09-844c-2cb8e2831890,c_69e05b80-7b62-4f8b-96f6-5a81a2810f49,ambre.thomas61291@msn.com,P,1,14.68,44.47,2017-05-24 16:13:33.090
1,s_a430364c-fcce-40d6-9846-8f5e68c97c56,c_69e05b80-7b62-4f8b-96f6-5a81a2810f49,ambre.thomas61291@msn.com,K,4,0.00,221.28,2017-10-01 12:00:50.451
2,s_f3971578-9f8c-4b49-a30a-6e350e30e680,c_69e05b80-7b62-4f8b-96f6-5a81a2810f49,ambre.thomas61291@msn.com,H,1,0.00,104.68,2017-06-07 21:37:35.472
3,s_01339e4e-55bc-47db-8817-616f12d0c0f0,c_69e05b80-7b62-4f8b-96f6-5a81a2810f49,ambre.thomas61291@msn.com,H,1,0.00,56.79,2019-06-08 01:42:13.704
4,s_68dd677f-e69c-4d67-a5f8-6c956dbefbed,c_69e05b80-7b62-4f8b-96f6-5a81a2810f49,ambre.thomas61291@msn.com,G,1,7.31,34.05,2017-09-17 11:25:16.760
...,...,...,...,...,...,...,...,...
42481,s_8f1b0f64-80a3-4da0-ac53-bd0af936bbbe,c_68625a41-94d8-4483-8e30-6e9efe3442ca,maxence_huet930@yahoo.com,D,2,0.00,101.88,2017-01-28 23:39:16.192
42482,s_51addb36-5aa3-4ee0-85e3-62a1929f68cf,c_68625a41-94d8-4483-8e30-6e9efe3442ca,maxence_huet930@yahoo.com,J,1,0.00,39.70,2018-10-13 13:18:17.336
42483,s_15c579b5-8752-4a52-81bb-df4d12b9fecf,c_68625a41-94d8-4483-8e30-6e9efe3442ca,maxence_huet930@yahoo.com,U,4,0.00,345.32,2018-10-01 06:23:35.174
42484,s_0eb187e4-ddcc-4edc-b7be-03d479b144a1,c_68625a41-94d8-4483-8e30-6e9efe3442ca,maxence_huet930@yahoo.com,P,1,0.00,41.28,2018-11-14 10:18:31.601


In [ ]:
df_offline_orders_enriched = sqldf(offline_orders_enriched, globals())

In [ ]:
test = """
select
  *
from df_offline_orders_enriched
where email is null
"""
sqldf(test, globals())

,id,client_id,email,store,quantity,discount,total_price,created
0,s_1ae5f15c-35dc-4018-a813-1b3d387b024b,c_61f37e09-21f4-4afd-8021-f0cd26329de0,None,B,1,0.0,129.56,2019-08-14 05:01:07.305
1,s_79f7141c-14d1-43f8-be17-297269bf87da,c_5d533f00-abcc-400e-aa17-0887631db068,None,G,1,0.0,50.77,2016-11-18 18:22:46.605
2,s_433b5fd6-df7d-4b55-b505-fa7f7bb9c729,c_f71fa137-a5a6-4676-8f91-8050a3582e34,None,I,1,0.0,49.00,2018-02-17 15:05:26.950
3,s_879af2be-a264-40b2-8452-16ee51cef3e2,c_452b5930-2f34-4686-b656-b5360bb9ad9b,None,H,1,0.0,59.90,2018-05-13 07:27:52.553
4,s_5fd81fc5-7a09-4216-9307-c8c59670a4b7,c_559531dc-8f02-4ac2-8d29-0084c451d511,None,T,1,0.0,77.77,2017-12-28 22:27:34.200
...,...,...,...,...,...,...,...,...
1443,s_9ad03e1a-21c8-45de-8864-70f3b2b2b341,c_d8a09f08-e4c2-43dd-90e3-34ec5bf32805,None,O,1,0.0,50.48,2016-07-15 10:23:05.830
1444,s_ed0fb7ef-0623-4eb0-8c77-c2e15a08167d,c_8ee6efca-44be-4961-baa6-1cbbd045d396,None,D,3,0.0,266.91,2016-09-15 18:53:28.205
1445,s_1df954cf-30b7-4883-9836-f38a54c50718,c_917edbc6-7b86-471e-9d51-a18afcb01943,None,E,2,0.0,140.90,2019-08-06 23:21:35.549
1446,s_58aea43c-234f-4f59-a496-1d89d702b264,c_419e7f01-b1ec-427d-aa35-b5d7059579f3,None,E,3,0.0,242.67,2016-01-17 07:03:28.782


We see here that some customer are registered with a contact_id in the offline order table but they don't have a row dedicated in the offline contact table (no contact infos).

#### Offline and online union


Now we can union the two order model. Here we also create a channel field to differenciate offline orders and online orders.

Here we create a field client_id that is a key to identify each client (by his email or his contact_id if there is no email associated to this client).

In [ ]:
orders = """
select
  id
  , case when email is not null then email
         else client_id end as client_id
  , email
  , store	
  , quantity	
  , discount	
  , total_price	
  , created
  , 'offline' as channel
from df_offline_orders_enriched

union

select
  id
  , email as client_id
  , email
  , 'website' as store	
  , quantity	
  , discount	
  , total_price	
  , created
  , 'online' as channel
from online_orders
"""
sqldf(orders, globals())

,id,client_id,email,store,quantity,discount,total_price,created,channel
0,s_000288b0-782d-47e1-b850-9ded39df59d0,hugo12715@sfr.fr,hugo12715@sfr.fr,S,1,0.00,77.85,2018-01-27 14:47:00.422,offline
1,s_000300f2-ea49-47fb-8b86-6d13973dbb4f,anas23226@live.fr,anas23226@live.fr,I,1,13.78,53.56,2018-06-09 09:47:58.001,offline
2,s_00035885-2c83-4bf5-98c2-b1c05e296aeb,manon.julien368@free.fr,manon.julien368@free.fr,L,4,24.15,152.37,2018-06-01 02:55:48.334,offline
3,s_0004c091-cca4-4910-b29f-4eb04f3ff5cc,dolorum.lola_gonzalez722@hotmail.fr,dolorum.lola_gonzalez722@hotmail.fr,S,1,0.00,68.57,2019-08-19 05:05:51.163,offline
4,s_0005a827-df8e-4715-85b4-c4b8040aaa56,libero.ocane1680@gmail.com,libero.ocane1680@gmail.com,F,1,0.00,58.28,2018-06-20 18:41:09.947,offline
...,...,...,...,...,...,...,...,...,...
69355,w_fff9c51f-6748-4757-a284-267b0550519f,et-julien_julien11697@hotmail.fr,et-julien_julien11697@hotmail.fr,website,1,0.00,81.53,2019-09-04 07:32:42.628,online
69356,w_fffbedb7-22ba-4636-883f-f3816499c267,ines99892@gmx.fr,ines99892@gmx.fr,website,1,0.00,81.61,2019-11-28 05:51:04.023,online
69357,w_fffe222e-386a-491c-9fbe-5c10a8c20628,cumque_benjamin_lemoine36635@yahoo.fr,cumque_benjamin_lemoine36635@yahoo.fr,website,1,0.00,51.42,2018-05-03 06:25:08.199,online
69358,w_fffedffd-114a-459c-addd-cf9a3ebb75b8,lina63137@laposte.net,lina63137@laposte.net,website,1,0.00,86.85,2019-10-14 13:56:16.579,online


In [ ]:
df_orders = sqldf(orders, globals())

Thanks to this, we can enrich the list of client_id (we will afterward enrich the list of contact that we have that is not the full list of customer):

In [ ]:
client_id_not_in_contact_list = """
select distinct
  client_id
  , min(created) as created
from df_orders
where email is null
group by 1
"""
sqldf(client_id_not_in_contact_list, globals())

,client_id,created
0,c_004958bb-c2a9-468a-9a5d-92d94f6e6000,2019-03-11 23:02:03.440
1,c_00661b87-dcb5-48aa-81b8-225e84cf2721,2017-02-26 12:26:54.246
2,c_00727e1b-1dd5-44c6-b5c2-2ffb941ac38e,2017-02-15 04:02:59.445
3,c_00738c15-14e2-4487-8147-05a0485d0ee5,2018-05-07 08:03:56.274
4,c_0131033e-77ca-4800-8be9-0f093806f4ee,2017-11-21 09:54:26.887
...,...,...
1443,c_ff9741b2-0ab3-4240-96c6-96fce155531f,2016-11-11 16:53:55.518
1444,c_ff9f0b4d-50b1-47f2-85af-5631f273fe49,2017-03-30 17:16:37.085
1445,c_ffabf067-d563-4921-9cd9-238c9c751377,2019-05-17 19:11:35.015
1446,c_ffb76458-9c40-4f4f-804c-31bfca8ca6d0,2019-03-09 15:04:10.547


In [ ]:
df_client_id_not_in_contact_list = sqldf(client_id_not_in_contact_list, globals())

# 4. DATA EXPLORATION & CLEANING


Now that we have all the data stored in 2 dataframes we're going to explore and clean them. 

The goal is to identify in these dataframes :    
- columns that are not usable for the analysis
- columns that need some modifications/reformatting to be used
- columns that are not relevant here

## Mobile, address and zip contact's column 

In addition to the contacts datas that we have it is interesting to add columns that will give us the information about the presence or absence of some datas. Especially for fields where we don't have much datas.

In [ ]:
offline_contacts.shape

(10908, 10)

In [ ]:
offline_contacts.isna().sum()

id              0
email           0
firstname    2225
lastname     2225
civility     6018
birth        5085
mobile       7256
address      8769
zip          8769
created         0
dtype: int64

Percentage of empty row for each field :

In [ ]:
offline_contacts.isna().sum()*100 / offline_contacts.shape[0]

id            0.000000
email         0.000000
firstname    20.397873
lastname     20.397873
civility     55.170517
birth        46.617162
mobile       66.519985
address      80.390539
zip          80.390539
created       0.000000
dtype: float64

However, the presence or the absence of these datas are good insight that we'll consider.

To extract information from these data we can use these conversions :
- birth --> has_given_age (0/1)

- mobile --> has_given_mobile_number (0/1)

- adress --> has_given_adress (0/1)

In [ ]:
contacts_1 = """
select
  email       
  , firstname     
  , lastname      
  , civility      
  , birth
  , case 
      when birth is null then 0
      else 1
      end as has_given_age
  , mobile
  , case 
      when mobile is null then 0
      else 1
      end as has_given_mobile_number
  , SUBSTR(zip,1,2) as department
  , case 
      when address is null then 0
      else 1
      end as has_given_address
  , created	
  , channel
  , is_client_using_both_channel	
  , optin
from df_contacts_and_consent
"""
sqldf(contacts_1, globals())

,email,firstname,lastname,civility,birth,has_given_age,mobile,has_given_mobile_number,department,has_given_address,created,channel,is_client_using_both_channel,optin
0,a-ethan.blanchard620@hotmail.fr,None,None,None,NaN,0,3.365230e+10,1,None,0,2019-08-08 14:54:33.270,offline,no,NaN
1,a-evan.roux29452@gmail.com,None,None,None,NaN,0,NaN,0,None,0,2019-04-25 22:05:31.708,offline,no,NaN
2,a-juliette.lopez61584@hotmail.fr,None,None,m,NaN,0,NaN,0,None,0,2017-07-18 14:35:34.433,online,yes,0.0
3,a-noah.fournier506@yahoo.fr,None,None,mlle,NaN,0,6.527320e+08,1,None,0,2015-06-01 22:32:18.820,offline,no,NaN
4,a.chlo.muller29774@hotmail.fr,None,None,Mr,19840619.0,1,NaN,0,None,0,2019-05-30 01:01:19.406,online,yes,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,zoe_remy383@yahoo.fr,None,None,None,19610809.0,1,3.364086e+10,1,None,0,2019-07-04 11:09:56.327,online,no,0.0
14996,zoe_robert59@laposte.net,zoe,robert,None,NaN,0,3.369949e+10,1,89,1,2019-10-22 09:45:25.102,offline,no,NaN
14997,zoe_roussel594@numericable.fr,zoe,roussel,None,NaN,0,NaN,0,None,0,2016-05-13 18:27:32.473,offline,no,NaN
14998,zoe_simon507@neuf.fr,None,None,None,19690623.0,1,NaN,0,None,0,2019-04-30 14:35:02.980,online,no,0.0


In [ ]:
df_contact_1 = sqldf(contacts_1, globals())

## Birth and created contact's columns

We can add some informations using available datas :

- birth --> calculate the age (in years), that will allow to do client bucket per age

- created --> to client_for x days, that will allow to do client bucket per client longevity

We will use the last created date to calculate the client_for_x_days field

In [ ]:
test =  """
select max(created) from df_contact_1
"""
sqldf(test, globals())

,max(created)
0,2019-12-30 23:09:09.110


In [ ]:
contacts_2 = """
select
  email        
  , firstname     
  , lastname      
  , civility      
  , strftime('%Y', datetime('now')) - cast(birth/10000 as int) as age
  , has_given_age
  , mobile
  , has_given_mobile_number
  , department
  , has_given_address
  , created
  , cast(JULIANDAY('2019-12-30 23:09:09.110') - JULIANDAY(created) as int)  as client_for_x_days
  , channel	
  , is_client_using_both_channel
  , optin
from df_contact_1
"""
sqldf(contacts_2, globals())

,email,firstname,lastname,civility,age,has_given_age,mobile,has_given_mobile_number,department,has_given_address,created,client_for_x_days,channel,is_client_using_both_channel,optin
0,a-ethan.blanchard620@hotmail.fr,None,None,None,NaN,0,3.365230e+10,1,None,0,2019-08-08 14:54:33.270,144,offline,no,NaN
1,a-evan.roux29452@gmail.com,None,None,None,NaN,0,NaN,0,None,0,2019-04-25 22:05:31.708,249,offline,no,NaN
2,a-juliette.lopez61584@hotmail.fr,None,None,m,NaN,0,NaN,0,None,0,2017-07-18 14:35:34.433,895,online,yes,0.0
3,a-noah.fournier506@yahoo.fr,None,None,mlle,NaN,0,6.527320e+08,1,None,0,2015-06-01 22:32:18.820,1673,offline,no,NaN
4,a.chlo.muller29774@hotmail.fr,None,None,Mr,38.0,1,NaN,0,None,0,2019-05-30 01:01:19.406,214,online,yes,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,zoe_remy383@yahoo.fr,None,None,None,61.0,1,3.364086e+10,1,None,0,2019-07-04 11:09:56.327,179,online,no,0.0
14996,zoe_robert59@laposte.net,zoe,robert,None,NaN,0,3.369949e+10,1,89,1,2019-10-22 09:45:25.102,69,offline,no,NaN
14997,zoe_roussel594@numericable.fr,zoe,roussel,None,NaN,0,NaN,0,None,0,2016-05-13 18:27:32.473,1326,offline,no,NaN
14998,zoe_simon507@neuf.fr,None,None,None,53.0,1,NaN,0,None,0,2019-04-30 14:35:02.980,244,online,no,0.0


In [ ]:
df_contacts_2 = sqldf(contacts_2, globals())

##Fields exploration

### Contacts

**Age :**

In [ ]:
df_contacts_2.age.describe()
# split in 3 adults categories is seem a good choice
# quartiles help to choose buckets

count    8663.000000
mean       41.369502
std        12.174083
min        21.000000
25%        31.000000
50%        42.000000
75%        52.000000
max        62.000000
Name: age, dtype: float64

**Client_for_x_days**

In [ ]:
df_contacts_2.client_for_x_days.describe()
# quartiles help to choose buckets

count    15000.000000
mean       657.695400
std        467.096346
min          0.000000
25%        262.000000
50%        579.000000
75%        983.000000
max       1824.000000
Name: client_for_x_days, dtype: float64

### Orders

**Discount**

In [ ]:
df_orders[df_orders.discount>0].discount.describe()
# quartiles help to choose buckets

count    13719.000000
mean        21.121623
std         17.939202
min          3.030000
25%          9.350000
50%         15.140000
75%         26.220000
max        147.070000
Name: discount, dtype: float64

**Quantity**

In [ ]:
df_orders.quantity.unique()
# 4 values no need for buckets

array([1, 4, 3, 2])

**Orders**

In [ ]:
df_orders.total_price.describe()
# quartiles help to choose buckets

count    69360.000000
mean       117.356785
std         95.711742
min         22.760000
25%         55.540000
50%         82.095000
75%        140.570000
max        599.920000
Name: total_price, dtype: float64

## Bucket creation for analysis purposes

As seen just before, some datas can be stored in bucket that's the case for the following datas (we used previous exploration and quartiles to define the buckets)
:     

**1. Contacts :**
- age : Young Adults (0-30) , Middle-aged Adults (31-50), Old-Adults (51+)
- client_for_x_days (5+ years, 2-5 years, 1-2 years, 6-12 months, 1-6 months, <1 month)

**2. Orders :**
- discount (1-10, 11-15, 16-25, 26-50,50+)
- total price (0-40, 41-100, 101-150, 200+)


### Contacts buckets

In [ ]:
contacts_w_buckets = """
select
  email      
  , firstname     
  , lastname      
  , civility      
  , age
  , case 
      when age <= 30 then 'Young Adults'
      when age > 30 and age <= 50 then 'Middle-aged Adults'
      when age > 50 then 'Old-Adults'
      else null
      end as age_bucket 
  , has_given_age
  , mobile
  , has_given_mobile_number
  , department
  , has_given_address
  , created
  , client_for_x_days
  , case 
      when client_for_x_days <= 30 then '<1 month'
      when client_for_x_days > 30 and client_for_x_days <= 180 then '1-6 months'
      when client_for_x_days > 180 and client_for_x_days <= 360 then '6-12 months'
      when client_for_x_days > 360 and client_for_x_days <= 720 then '1-2 years'
      when client_for_x_days > 720 and client_for_x_days <= 1800 then '2-5 years'
      else '5+ years'
      end as client_for_x_days_bucket
  , channel	
  , is_client_using_both_channel
  , optin
from df_contacts_2
"""
sqldf(contacts_w_buckets, globals())

,email,firstname,lastname,civility,age,age_bucket,has_given_age,mobile,has_given_mobile_number,department,has_given_address,created,client_for_x_days,client_for_x_days_bucket,channel,is_client_using_both_channel,optin
0,a-ethan.blanchard620@hotmail.fr,None,None,None,NaN,None,0,3.365230e+10,1,None,0,2019-08-08 14:54:33.270,144,1-6 months,offline,no,NaN
1,a-evan.roux29452@gmail.com,None,None,None,NaN,None,0,NaN,0,None,0,2019-04-25 22:05:31.708,249,6-12 months,offline,no,NaN
2,a-juliette.lopez61584@hotmail.fr,None,None,m,NaN,None,0,NaN,0,None,0,2017-07-18 14:35:34.433,895,2-5 years,online,yes,0.0
3,a-noah.fournier506@yahoo.fr,None,None,mlle,NaN,None,0,6.527320e+08,1,None,0,2015-06-01 22:32:18.820,1673,2-5 years,offline,no,NaN
4,a.chlo.muller29774@hotmail.fr,None,None,Mr,38.0,Middle-aged Adults,1,NaN,0,None,0,2019-05-30 01:01:19.406,214,6-12 months,online,yes,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,zoe_remy383@yahoo.fr,None,None,None,61.0,Old-Adults,1,3.364086e+10,1,None,0,2019-07-04 11:09:56.327,179,1-6 months,online,no,0.0
14996,zoe_robert59@laposte.net,zoe,robert,None,NaN,None,0,3.369949e+10,1,89,1,2019-10-22 09:45:25.102,69,1-6 months,offline,no,NaN
14997,zoe_roussel594@numericable.fr,zoe,roussel,None,NaN,None,0,NaN,0,None,0,2016-05-13 18:27:32.473,1326,2-5 years,offline,no,NaN
14998,zoe_simon507@neuf.fr,None,None,None,53.0,Old-Adults,1,NaN,0,None,0,2019-04-30 14:35:02.980,244,6-12 months,online,no,0.0


In [ ]:
df_contacts_w_buckets = sqldf(contacts_w_buckets, globals())

### Orders buckets

In [ ]:
orders_w_buckets = """
select
  id
  , client_id
  , email
  , store	
  , quantity	
  , discount	
  , case 
      when discount = 0 then 'No discount'
      when discount > 0 and discount <= 10 then '1-10%'
      when discount > 10 and discount <= 15 then '11-15%'
      when discount > 15 and discount <= 25 then '16-25%'
      when discount > 25 and discount <= 50 then '16-25%'
      else '50%+'
      end as discount_bucket 
  , total_price	
  , case 
      when total_price <= 40 then '0-40$'
      when total_price > 40 and total_price <= 100 then '40-100$'
      when total_price > 100 and total_price <= 150 then '101-150$'
      else '200$+'
      end as total_price_bucket 
  , created
  , channel
from df_orders
"""
sqldf(orders_w_buckets, globals())

,id,client_id,email,store,quantity,discount,discount_bucket,total_price,total_price_bucket,created,channel
0,s_000288b0-782d-47e1-b850-9ded39df59d0,hugo12715@sfr.fr,hugo12715@sfr.fr,S,1,0.00,No discount,77.85,40-100$,2018-01-27 14:47:00.422,offline
1,s_000300f2-ea49-47fb-8b86-6d13973dbb4f,anas23226@live.fr,anas23226@live.fr,I,1,13.78,11-15%,53.56,40-100$,2018-06-09 09:47:58.001,offline
2,s_00035885-2c83-4bf5-98c2-b1c05e296aeb,manon.julien368@free.fr,manon.julien368@free.fr,L,4,24.15,16-25%,152.37,200$+,2018-06-01 02:55:48.334,offline
3,s_0004c091-cca4-4910-b29f-4eb04f3ff5cc,dolorum.lola_gonzalez722@hotmail.fr,dolorum.lola_gonzalez722@hotmail.fr,S,1,0.00,No discount,68.57,40-100$,2019-08-19 05:05:51.163,offline
4,s_0005a827-df8e-4715-85b4-c4b8040aaa56,libero.ocane1680@gmail.com,libero.ocane1680@gmail.com,F,1,0.00,No discount,58.28,40-100$,2018-06-20 18:41:09.947,offline
...,...,...,...,...,...,...,...,...,...,...,...
69355,w_fff9c51f-6748-4757-a284-267b0550519f,et-julien_julien11697@hotmail.fr,et-julien_julien11697@hotmail.fr,website,1,0.00,No discount,81.53,40-100$,2019-09-04 07:32:42.628,online
69356,w_fffbedb7-22ba-4636-883f-f3816499c267,ines99892@gmx.fr,ines99892@gmx.fr,website,1,0.00,No discount,81.61,40-100$,2019-11-28 05:51:04.023,online
69357,w_fffe222e-386a-491c-9fbe-5c10a8c20628,cumque_benjamin_lemoine36635@yahoo.fr,cumque_benjamin_lemoine36635@yahoo.fr,website,1,0.00,No discount,51.42,40-100$,2018-05-03 06:25:08.199,online
69358,w_fffedffd-114a-459c-addd-cf9a3ebb75b8,lina63137@laposte.net,lina63137@laposte.net,website,1,0.00,No discount,86.85,40-100$,2019-10-14 13:56:16.579,online


In [ ]:
df_orders_w_buckets = sqldf(orders_w_buckets, globals())

## Gender data enrichment

Looking at the dataset in detail, we can see that the civility column is not usable as it is for multiple reasons :     
- some customer seem to register the wrong gender, for instance there is théo	dupont that filled 'mle' in the civility column. There are a lot of other similar example proving that the data of this column is not trustable
- the format is not uniform. A lot of customer just put 'M' and we cannot know if this M stand for Mr or Mlle or Mme.

A solution to still have gender information would be to use a Machine Learning or other any algorithm to classify the gender of each customer based on their first name.

In [ ]:
df_contacts_w_buckets.tail()

,email,firstname,lastname,civility,age,age_bucket,has_given_age,mobile,has_given_mobile_number,department,has_given_address,created,client_for_x_days,client_for_x_days_bucket,channel,is_client_using_both_channel,optin
14995,zoe_remy383@yahoo.fr,None,None,None,61.0,Old-Adults,1,3.364086e+10,1,None,0,2019-07-04 11:09:56.327,179,1-6 months,online,no,0.0
14996,zoe_robert59@laposte.net,zoe,robert,None,NaN,None,0,3.369949e+10,1,89,1,2019-10-22 09:45:25.102,69,1-6 months,offline,no,NaN
14997,zoe_roussel594@numericable.fr,zoe,roussel,None,NaN,None,0,NaN,0,None,0,2016-05-13 18:27:32.473,1326,2-5 years,offline,no,NaN
14998,zoe_simon507@neuf.fr,None,None,None,53.0,Old-Adults,1,NaN,0,None,0,2019-04-30 14:35:02.980,244,6-12 months,online,no,0.0
14999,zoe_vincent356@numericable.fr,zoe,vincent,None,31.0,Middle-aged Adults,1,NaN,0,None,0,2019-10-07 07:27:29.997,84,1-6 months,online,yes,0.0


First, we need the list of first name :

In [ ]:
first_name_list = list(df_contacts_w_buckets.firstname.unique()[1:])
print(first_name_list)

['adam', 'adrien', 'alexandre', 'alexis', 'alice', 'alicia', 'ambre', 'anaïs', 'antoine', 'arthur', 'axel', 'baptiste', 'benjamin', 'camille', 'carla', 'charlotte', 'chloé', 'clara', 'célia', 'clémence', 'clément', 'elisa', 'emilie', 'enzo', 'ethan', 'eva', 'evan', 'gabriel', 'hugo', 'ines', 'jade', 'jeanne', 'jules', 'julie', 'julien', 'juliette', 'justine', 'kylian', 'léa', 'laura', 'lena', 'lilou', 'lina', 'lisa', 'léo', 'lola', 'lou', 'louis', 'louise', 'louna', 'lucas', 'lucie', 'mael', 'maeva', 'maëlle', 'maëlys', 'manon', 'marie', 'mathilde', 'mathis', 'mathéo', 'mattéo', 'maxence', 'maxime', 'mélissa', 'mohamed', 'nathan', 'nicolas', 'noa', 'noah', 'noémie', 'océane', 'paul', 'pauline', 'pierre', 'quentin', 'raphaël', 'rayan', 'romain', 'romane', 'sacha', 'sarah', 'théo', 'thomas', 'tom', 'valentin', 'victor', 'yanis', 'zoe']


Then we can use the API genderize (see more about it [here](https://gender-api.com/fr?utm_source=google&utm_medium=searcz&utm_campaign=genderize&price-set=UMG&gclid=CjwKCAjwpqCZBhAbEiwAa7pXefb7eWmlekDycFuczjkc33CuEWkVlOtv7t6JwAGf_-3P7Y1y1BrYwhoCbpEQAvD_BwE)) and apply it to each name of the list.

This give us a list of gender for each name :

In [ ]:
import requests, json
  
gender_list = []
for i in range (0,len(first_name_list)) :
  name = first_name_list[i]
  content = requests.get(f"https://api.genderize.io?name={name}").text
  gender = json.loads(content)['gender']
  gender_list.append(gender)
print(gender_list)

['male', 'male', 'male', 'male', 'female', 'female', 'female', 'female', 'male', 'male', 'male', 'male', 'male', 'female', 'female', 'female', None, 'female', None, 'female', None, 'female', 'female', 'male', 'male', 'female', 'male', 'male', 'male', 'female', 'female', 'female', 'male', 'female', 'male', 'female', 'female', 'male', None, 'female', 'female', 'female', 'female', 'female', None, 'female', 'female', 'male', 'female', 'female', 'male', 'female', 'male', 'female', None, None, 'female', 'female', 'female', 'male', None, None, 'male', 'male', 'female', 'male', 'male', 'male', 'female', 'male', 'female', None, 'male', 'female', 'male', 'male', 'male', 'male', 'male', 'female', 'male', 'female', 'male', 'male', 'male', 'male', 'male', 'male', 'female']


Then we create a dataframe using the gender list and the first name list :

In [ ]:
d = {'firstname':first_name_list,'Gender':gender_list}
df_gender = pd.DataFrame(d)
df_gender.head()

,firstname,Gender
0,adam,male
1,adrien,male
2,alexandre,male
3,alexis,male
4,alice,female


And finally we :
- join the gender from this dataframe to the contact model (and use the gender column in place of the civility one) 
- add the client_id_not_in_contact_list table to create the final table with the full list of customer


In [ ]:
contacts_final = """
select
  email as client_id
  , email    
  , c.firstname     
  , lastname      
  , g.gender      
  , age
  , age_bucket 
  , has_given_age
  , mobile
  , has_given_mobile_number
  , department
  , has_given_address
  , created
  , client_for_x_days
  , client_for_x_days_bucket
  , channel	
  , is_client_using_both_channel
  , optin
from df_contacts_w_buckets as c
left join df_gender as g
  using(firstname)

union

select    
  client_id
  , null as email    
  , null as firstname     
  , null as lastname      
  , null as gender      
  , null as age
  , null as age_bucket 
  , 0 as has_given_age
  , null as mobile
  , 0 as has_given_mobile_number
  , null as department
  , 0 as has_given_address
  , created 
  , cast(JULIANDAY('2019-12-30 23:09:09.110') - JULIANDAY(created) as int)  as client_for_x_days 
  , case 
      when cast(JULIANDAY('2019-12-30 23:09:09.110') - JULIANDAY(created) as int) <= 30 then '<1 month'
      when cast(JULIANDAY('2019-12-30 23:09:09.110') - JULIANDAY(created) as int) > 30 and cast(JULIANDAY('2019-12-30 23:09:09.110') - JULIANDAY(created) as int) <= 180 then '1-6 months'
      when cast(JULIANDAY('2019-12-30 23:09:09.110') - JULIANDAY(created) as int) > 180 and cast(JULIANDAY('2019-12-30 23:09:09.110') - JULIANDAY(created) as int) <= 360 then '6-12 months'
      when cast(JULIANDAY('2019-12-30 23:09:09.110') - JULIANDAY(created) as int) > 360 and cast(JULIANDAY('2019-12-30 23:09:09.110') - JULIANDAY(created) as int) <= 720 then '1-2 years'
      when cast(JULIANDAY('2019-12-30 23:09:09.110') - JULIANDAY(created) as int) > 720 and cast(JULIANDAY('2019-12-30 23:09:09.110') - JULIANDAY(created) as int) <= 1800 then '2-5 years'
      else '5+ years'
      end as client_for_x_days_bucket
  , 'offline' as channel	
  , 'no' is_client_using_both_channel
  , null as optin
from df_client_id_not_in_contact_list

"""
sqldf(contacts_final, globals())



,client_id,email,firstname,lastname,Gender,age,age_bucket,has_given_age,mobile,has_given_mobile_number,department,has_given_address,created,client_for_x_days,client_for_x_days_bucket,channel,is_client_using_both_channel,optin
0,a-ethan.blanchard620@hotmail.fr,a-ethan.blanchard620@hotmail.fr,None,None,None,NaN,None,0,3.365230e+10,1,None,0,2019-08-08 14:54:33.270,144,1-6 months,offline,no,NaN
1,a-evan.roux29452@gmail.com,a-evan.roux29452@gmail.com,None,None,None,NaN,None,0,NaN,0,None,0,2019-04-25 22:05:31.708,249,6-12 months,offline,no,NaN
2,a-juliette.lopez61584@hotmail.fr,a-juliette.lopez61584@hotmail.fr,None,None,None,NaN,None,0,NaN,0,None,0,2017-07-18 14:35:34.433,895,2-5 years,online,yes,0.0
3,a-noah.fournier506@yahoo.fr,a-noah.fournier506@yahoo.fr,None,None,None,NaN,None,0,6.527320e+08,1,None,0,2015-06-01 22:32:18.820,1673,2-5 years,offline,no,NaN
4,a.chlo.muller29774@hotmail.fr,a.chlo.muller29774@hotmail.fr,None,None,None,38.0,Middle-aged Adults,1,NaN,0,None,0,2019-05-30 01:01:19.406,214,6-12 months,online,yes,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16443,zoe_remy383@yahoo.fr,zoe_remy383@yahoo.fr,None,None,None,61.0,Old-Adults,1,3.364086e+10,1,None,0,2019-07-04 11:09:56.327,179,1-6 months,online,no,0.0
16444,zoe_robert59@laposte.net,zoe_robert59@laposte.net,zoe,robert,female,NaN,None,0,3.369949e+10,1,89,1,2019-10-22 09:45:25.102,69,1-6 months,offline,no,NaN
16445,zoe_roussel594@numericable.fr,zoe_roussel594@numericable.fr,zoe,roussel,female,NaN,None,0,NaN,0,None,0,2016-05-13 18:27:32.473,1326,2-5 years,offline,no,NaN
16446,zoe_simon507@neuf.fr,zoe_simon507@neuf.fr,None,None,None,53.0,Old-Adults,1,NaN,0,None,0,2019-04-30 14:35:02.980,244,6-12 months,online,no,0.0


In [ ]:
df_contacts_final = sqldf(contacts_final, globals())

It's possible to improve this gender assigntation by using the email address and not the first name (email is given by all the contact and first name is not).

## Order model enriched by contact datas

I join the two models using the email. I assume that one email correspond to one customer (no customer with similar email for instance)

In [ ]:
orders_enriched = """
select
  o.id
  , o.client_id
  , o.email
  , firstname
  , lastname
  , o.store	
  , o.quantity	
  , o.discount	
  , discount_bucket 
  , total_price	
  , total_price_bucket 
  , o.created
  , o.channel
  , is_client_using_both_channel
  , c.gender
  , c.age
  , age_bucket
  , has_given_age
  , mobile
  , has_given_mobile_number
  , department	
  , has_given_address	
  , client_for_x_days	
  , client_for_x_days_bucket		
  , optin
from df_orders_w_buckets as o 
left join df_contacts_final as c
  on o.email = c.email

"""
sqldf(orders_enriched, globals())

,id,client_id,email,firstname,lastname,store,quantity,discount,discount_bucket,total_price,...,age,age_bucket,has_given_age,mobile,has_given_mobile_number,department,has_given_address,client_for_x_days,client_for_x_days_bucket,optin
0,s_000288b0-782d-47e1-b850-9ded39df59d0,hugo12715@sfr.fr,hugo12715@sfr.fr,hugo,lambert,S,1,0.00,No discount,77.85,...,NaN,None,0.0,6.424805e+08,1.0,89,1.0,771.0,2-5 years,NaN
1,s_000300f2-ea49-47fb-8b86-6d13973dbb4f,anas23226@live.fr,anas23226@live.fr,anaïs,maillard,I,1,13.78,11-15%,53.56,...,23.0,Young Adults,1.0,NaN,0.0,99,1.0,569.0,1-2 years,NaN
2,s_00035885-2c83-4bf5-98c2-b1c05e296aeb,manon.julien368@free.fr,manon.julien368@free.fr,manon,julien,L,4,24.15,16-25%,152.37,...,24.0,Young Adults,1.0,NaN,0.0,None,0.0,577.0,1-2 years,NaN
3,s_0004c091-cca4-4910-b29f-4eb04f3ff5cc,dolorum.lola_gonzalez722@hotmail.fr,dolorum.lola_gonzalez722@hotmail.fr,None,None,S,1,0.00,No discount,68.57,...,43.0,Middle-aged Adults,1.0,NaN,0.0,None,0.0,133.0,1-6 months,0.0
4,s_0005a827-df8e-4715-85b4-c4b8040aaa56,libero.ocane1680@gmail.com,libero.ocane1680@gmail.com,None,None,F,1,0.00,No discount,58.28,...,NaN,None,0.0,NaN,0.0,None,0.0,1122.0,2-5 years,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69355,w_fff9c51f-6748-4757-a284-267b0550519f,et-julien_julien11697@hotmail.fr,et-julien_julien11697@hotmail.fr,None,None,website,1,0.00,No discount,81.53,...,46.0,Middle-aged Adults,1.0,3.361955e+10,1.0,None,0.0,186.0,6-12 months,0.0
69356,w_fffbedb7-22ba-4636-883f-f3816499c267,ines99892@gmx.fr,ines99892@gmx.fr,None,None,website,1,0.00,No discount,81.61,...,38.0,Middle-aged Adults,1.0,NaN,0.0,None,0.0,39.0,1-6 months,1.0
69357,w_fffe222e-386a-491c-9fbe-5c10a8c20628,cumque_benjamin_lemoine36635@yahoo.fr,cumque_benjamin_lemoine36635@yahoo.fr,None,None,website,1,0.00,No discount,51.42,...,NaN,None,0.0,3.369053e+10,1.0,None,0.0,1306.0,2-5 years,1.0
69358,w_fffedffd-114a-459c-addd-cf9a3ebb75b8,lina63137@laposte.net,lina63137@laposte.net,None,None,website,1,0.00,No discount,86.85,...,58.0,Old-Adults,1.0,3.378742e+10,1.0,None,0.0,258.0,6-12 months,0.0


In [ ]:
df_orders_enriched = sqldf(orders_enriched, globals())

In [ ]:
df_orders_enriched.client_id.nunique()

16448

### New visitor and returning visitor information

Adding new visitor and returning visitor information over each order :

In [ ]:
df_orders_enriched = df_orders_enriched.sort_values(by='created')
df_orders_enriched['order_number']=df_orders_enriched.groupby(['client_id'])['created'].cumcount()+1
df_orders_enriched

,id,client_id,email,firstname,lastname,store,quantity,discount,discount_bucket,total_price,...,age_bucket,has_given_age,mobile,has_given_mobile_number,department,has_given_address,client_for_x_days,client_for_x_days_bucket,optin,order_number
64066,w_cc8669d9-8976-47e8-a30b-d005ae80e505,tom_lecomte231@hotmail.com,tom_lecomte231@hotmail.com,tom,lecomte,website,1,0.00,No discount,66.39,...,Middle-aged Adults,1.0,NaN,0.0,None,0.0,1824.0,5+ years,NaN,1
42749,w_0298e32f-408c-4a0f-81d1-59b78a12050d,ipsum.raphal_carpentier329@hotmail.fr,ipsum.raphal_carpentier329@hotmail.fr,None,None,website,1,8.89,1-10%,47.12,...,None,0.0,3.363748e+10,1.0,None,0.0,1824.0,5+ years,1.0,1
69217,w_fee2b2b4-9bcd-418a-8c02-283f36c7e26a,adrien_renault32742@laposte.net,adrien_renault32742@laposte.net,None,None,website,4,0.00,No discount,345.20,...,Young Adults,1.0,NaN,0.0,None,0.0,1824.0,5+ years,1.0,1
31282,s_bbd10765-1ffc-4fcd-a2c0-9784fea4f069,clara43650@live.fr,clara43650@live.fr,clara,gautier,H,1,0.00,No discount,36.71,...,Middle-aged Adults,1.0,NaN,0.0,36,1.0,1824.0,5+ years,1.0,1
26228,s_9dded997-9182-4e3d-92c8-023696c69bc0,vel.clmence_roger893@yahoo.fr,vel.clmence_roger893@yahoo.fr,None,None,S,4,33.56,16-25%,121.96,...,Middle-aged Adults,1.0,3.367958e+10,1.0,None,0.0,1823.0,5+ years,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67733,w_f087c544-468a-4e34-bed2-6b9d77a71a42,yanis_robert405@msn.com,yanis_robert405@msn.com,None,None,website,1,0.00,No discount,76.76,...,None,0.0,NaN,0.0,None,0.0,1.0,<1 month,0.0,6
35443,s_d5720091-45c2-42d7-9b29-eccb2bddfc5c,jade_prevost77849@free.fr,jade_prevost77849@free.fr,jade,prevost,M,1,0.00,No discount,31.75,...,None,0.0,NaN,0.0,32,1.0,19.0,<1 month,NaN,6
1936,s_0c0c3e97-6f54-4bdc-a9cf-49c1713dd908,sit.laura_mercier616@hotmail.fr,sit.laura_mercier616@hotmail.fr,None,None,U,4,0.00,No discount,195.28,...,None,0.0,3.374580e+10,1.0,None,0.0,1.0,<1 month,NaN,6
53611,w_69097c5f-c259-449d-be6b-f6cae4ef34d0,mathilde96232@gmail.com,mathilde96232@gmail.com,mathilde,lucas,website,2,15.38,16-25%,75.48,...,None,0.0,NaN,0.0,None,0.0,3.0,<1 month,1.0,3


Example :

In [ ]:
test = """
select
client_id,
created,
order_number
from df_orders_enriched
where client_id = 'hugo12715@sfr.fr'
"""
sqldf(test, globals())

,client_id,created,order_number
0,hugo12715@sfr.fr,2017-11-19 14:11:59.555,1
1,hugo12715@sfr.fr,2018-01-21 03:52:29.486,2
2,hugo12715@sfr.fr,2018-01-27 14:47:00.422,3
3,hugo12715@sfr.fr,2019-06-08 10:16:16.691,4
4,hugo12715@sfr.fr,2019-10-21 18:05:47.858,5


In [ ]:
nb_of_orders = """
select
client_id
, max(order_number) as nb_of_orders
from df_orders_enriched
group by client_id
"""
sqldf(nb_of_orders, globals())

,client_id,nb_of_orders
0,a-ethan.blanchard620@hotmail.fr,4
1,a-evan.roux29452@gmail.com,3
2,a-juliette.lopez61584@hotmail.fr,5
3,a-noah.fournier506@yahoo.fr,3
4,a.chlo.muller29774@hotmail.fr,3
...,...,...
16443,zoe_remy383@yahoo.fr,3
16444,zoe_robert59@laposte.net,6
16445,zoe_roussel594@numericable.fr,5
16446,zoe_simon507@neuf.fr,4


In [ ]:
df_nb_of_orders = sqldf(nb_of_orders, globals())

In [ ]:
df_orders_final = """
select
*
, case when order_number = 1 then 'new visitor'
     else 'returning visitor' 
     end as visitor_type
from df_orders_enriched as o 
left join df_nb_of_orders as n 
using (client_id)
"""
sqldf(df_orders_final, globals())

,id,client_id,email,firstname,lastname,store,quantity,discount,discount_bucket,total_price,...,mobile,has_given_mobile_number,department,has_given_address,client_for_x_days,client_for_x_days_bucket,optin,order_number,nb_of_orders,visitor_type
0,w_cc8669d9-8976-47e8-a30b-d005ae80e505,tom_lecomte231@hotmail.com,tom_lecomte231@hotmail.com,tom,lecomte,website,1,0.00,No discount,66.39,...,NaN,0.0,None,0.0,1824.0,5+ years,NaN,1,7,new visitor
1,w_0298e32f-408c-4a0f-81d1-59b78a12050d,ipsum.raphal_carpentier329@hotmail.fr,ipsum.raphal_carpentier329@hotmail.fr,None,None,website,1,8.89,1-10%,47.12,...,3.363748e+10,1.0,None,0.0,1824.0,5+ years,1.0,1,6,new visitor
2,w_fee2b2b4-9bcd-418a-8c02-283f36c7e26a,adrien_renault32742@laposte.net,adrien_renault32742@laposte.net,None,None,website,4,0.00,No discount,345.20,...,NaN,0.0,None,0.0,1824.0,5+ years,1.0,1,5,new visitor
3,s_bbd10765-1ffc-4fcd-a2c0-9784fea4f069,clara43650@live.fr,clara43650@live.fr,clara,gautier,H,1,0.00,No discount,36.71,...,NaN,0.0,36,1.0,1824.0,5+ years,1.0,1,5,new visitor
4,s_9dded997-9182-4e3d-92c8-023696c69bc0,vel.clmence_roger893@yahoo.fr,vel.clmence_roger893@yahoo.fr,None,None,S,4,33.56,16-25%,121.96,...,3.367958e+10,1.0,None,0.0,1823.0,5+ years,NaN,1,5,new visitor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69355,w_f087c544-468a-4e34-bed2-6b9d77a71a42,yanis_robert405@msn.com,yanis_robert405@msn.com,None,None,website,1,0.00,No discount,76.76,...,NaN,0.0,None,0.0,1.0,<1 month,0.0,6,6,returning visitor
69356,s_d5720091-45c2-42d7-9b29-eccb2bddfc5c,jade_prevost77849@free.fr,jade_prevost77849@free.fr,jade,prevost,M,1,0.00,No discount,31.75,...,NaN,0.0,32,1.0,19.0,<1 month,NaN,6,6,returning visitor
69357,s_0c0c3e97-6f54-4bdc-a9cf-49c1713dd908,sit.laura_mercier616@hotmail.fr,sit.laura_mercier616@hotmail.fr,None,None,U,4,0.00,No discount,195.28,...,3.374580e+10,1.0,None,0.0,1.0,<1 month,NaN,6,6,returning visitor
69358,w_69097c5f-c259-449d-be6b-f6cae4ef34d0,mathilde96232@gmail.com,mathilde96232@gmail.com,mathilde,lucas,website,2,15.38,16-25%,75.48,...,NaN,0.0,None,0.0,3.0,<1 month,1.0,3,3,returning visitor


In [ ]:
df_orders_final = sqldf(df_orders_final, globals())

Test and repartitionof the orders :

In [ ]:
df_test = """
select
order_number
, count(*)
from  df_orders_final
group by 1
"""
sqldf(df_test, globals())

,order_number,count(*)
0,1,16448
1,2,15000
2,3,15000
3,4,11380
4,5,7621
5,6,3825
6,7,86


In [ ]:
df_orders_final.columns

Index(['id', 'client_id', 'email', 'firstname', 'lastname', 'store',
       'quantity', 'discount', 'discount_bucket', 'total_price',
       'total_price_bucket', 'created', 'channel',
       'is_client_using_both_channel', 'Gender', 'age', 'age_bucket',
       'has_given_age', 'mobile', 'has_given_mobile_number', 'department',
       'has_given_address', 'client_for_x_days', 'client_for_x_days_bucket',
       'optin', 'order_number', 'nb_of_orders', 'visitor_type'],
      dtype='object')

# 5. EXPORT TO CSV

Finally, we will keep one data model that we're going to use for the visualisation :
- df_contacts_final (15000 rows × 16 columns)
- df_orders_final (69360 rows × 28 columns)

Dataframe to csv :

In [ ]:
from google.colab import files

df_orders_final.to_csv('df_orders_final.csv') 
files.download('df_orders_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_contacts_final.to_csv('df_contacts_final.csv') 
files.download('df_contacts_final.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>